In [2]:
using Distributions
using Optim

In [41]:
function generate_obs(n, m, params; verbose=true, select_eq=risk_dom)
    
    #set up params
    delta = params[1:n]
    beta = params[n+1:2*n]    
    
    x= rand(Normal(0.0, 1.0),2)
    e = rand(Logistic(),2)
    
    a = zeros(n,2)
    a[:,2] = m .- ones(n)
    a_next = zeros(n,2)
    a_next[:,2] = m .- ones(n)

    eq = false
    u_lower = zeros(n)
    u_upper = zeros(n)
   
    #iterative best responses
    while !eq
        eq = true
        #compute upper and lower bounds on utility...
        u_lower = x .+ e
        u_upper = u_lower[:]
        #compute correct bounds for each player
        for i=1:n
            for j=1:(n-1)
                j_index = mod(i+j-1,n)+1
                u_upper[i] = u_upper[i] + beta[i]*maximum(a[i,:]) + delta[i]*minimum(a[j_index,:])
                u_lower[i] = u_lower[i] + beta[i]*minimum(a[i,:]) + delta[i]*maximum(a[j_index,:])
            end
            
            #check to see if low is dominated
            if (u_lower[i] >= 0) & ( a_next[i,1] != a_next[i,2])
                a_next[i,1] = a_next[i,1] + 1
            end
            #check if high is dominated    
            if (u_upper[i] <= 0) & ( a_next[i,1] != a_next[i,2])
                a_next[i,2] = a_next[i,2] - 1
            end
            
            if verbose
                println(i," ", a[i,:]," ", a_next[i,:]," ", (a_next[i,:] == a[i,:]),  " ", eq)
                println(i," ", u_lower[i]," ", u_upper[i])
                println(a," ",a_next)
            end
            eq = eq & (a_next[i,:] == a[i,:])
        end
        
        if verbose
            println("----")
        end
        println("<<<---->>>")
        println(a," ",a_next)
        a =  a_next[:,:]
        println(a," ",a_next)
        println("<<<---->>>")
    end
    
    #reshape a correctly/select equilibria if mult
    a_return = select_eq(n, m, params, a, x, e)
    return a_return,x
end


function gen_data(N;n=2,m=3,select_eq=risk_dom,params= [-3,-3.5, .6, .7])
    Y = zeros(n,N)
    X = zeros(n,N)
    for i=1:N
        y,x = generate_obs(n,m,params;select_eq=select_eq)
        Y[:,i] = y
        X[:,i] = x
    end
    return Y,X
end


gen_data(1;n=2,m=3,params=[-1,-1,.9, .9])

1 [0.0, 2.0] [0.0, 2.0] true true
1 -2.1192982412178267 1.6807017587821733
[0.0 2.0; 0.0 2.0] [0.0 2.0; 0.0 2.0]
2 [0.0, 2.0] [1.0, 2.0] false true
2 0.6631757265764211 4.463175726576421
[0.0 2.0; 0.0 2.0] [0.0 2.0; 1.0 2.0]
----
<<<---->>>
[0.0 2.0; 0.0 2.0] [0.0 2.0; 1.0 2.0]
[0.0 2.0; 1.0 2.0] [0.0 2.0; 1.0 2.0]
<<<---->>>
1 [0.0, 2.0] [0.0, 2.0] true true
1 -2.1192982412178267 0.6807017587821733
[0.0 2.0; 1.0 2.0] [0.0 2.0; 1.0 2.0]
2 [1.0, 2.0] [2.0, 2.0] false true
2 1.563175726576421 4.463175726576421
[0.0 2.0; 1.0 2.0] [0.0 2.0; 2.0 2.0]
----
<<<---->>>
[0.0 2.0; 1.0 2.0] [0.0 2.0; 2.0 2.0]
[0.0 2.0; 2.0 2.0] [0.0 2.0; 2.0 2.0]
<<<---->>>
1 [0.0, 2.0] [0.0, 1.0] false true
1 -2.1192982412178267 -0.3192982412178267
[0.0 2.0; 2.0 2.0] [0.0 1.0; 2.0 2.0]
2 [2.0, 2.0] [2.0, 2.0] true false
2 2.463175726576421 4.463175726576421
[0.0 2.0; 2.0 2.0] [0.0 1.0; 2.0 2.0]
----
<<<---->>>
[0.0 2.0; 2.0 2.0] [0.0 1.0; 2.0 2.0]
[0.0 1.0; 2.0 2.0] [0.0 1.0; 2.0 2.0]
<<<---->>>
1 [0.0, 1.0] [0.

([0.0; 0.0], [1.53117; -0.286444])

In [ ]:
function enum_eq(a)
    #enumerate all the eq in the lattice
    
    #delete those that violate the bounds
    
end

enum_eq([0 2; 0 2; 0 0])

In [ ]:
function risk_dom(n, params, a, x, e)
    
    #set up params
    delta = params[1:n]
    beta = params[n+1:2*n] 
    
    #setup payoffs
    u_lower = beta.*x  .+ e
    u_upper = u_lower[:] .+ delta
    
    #set up a
    a_return = zeros(n)
    
    #enumerate the mult eq
    entrants = 0
    for i=1:n
        if (length(a[i])>1)
            entrants = entrants +1
        end
    end
    
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
        if (length(a[i])>1) & (select == i)
            println("random select: ", select, " ", i)
            a_return[i] = 1 .- a[i][1]
        end
    end
    
    if length(a[1])>1
        println(select, " ", a)
        println(u_lower, " ", u_upper)
        println(a_return)
    end
    
    return a_return
end

risk_dom(3,  [-3,-3.5,-4, .6, .7, .8], [[0,1],[0,1],[1]], [1,2,3], [-1,-2,-3])

In [ ]:
function pareto_opt(n, params, a, x, e; rho = -2)
    print("yo")
    a_return = zeros(n)
    lambda = exp(rho)/(1+exp(rho))
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
       
        if length(a[i])>1 & (select ==i)
            a_return[i] = 1 .- a[i][1]
        end
    end
    return a_return
end